## DES

In [2]:
!pip install pyDes

  Created wheel for pyDes: filename=pyDes-2.0.1-py2.py3-none-any.whl size=9566 sha256=2f9c01226d00e088c39a47d37333e2dbb07239bdd7339841de4a7163543bf5a6
  Stored in directory: /root/.cache/pip/wheels/57/45/3a/cd3cffc8044aa6b8670109a57db784cc7c64dc52259a11575a
Successfully built pyDes


In [3]:
import pyDes
from time import time

# For Python3, you'll need to use bytes, i.e.:
#key  -> Bytes containing the encryption key. 8 bytes for DES, 16 or 24 bytes
#	 for Triple DES
#mode -> Optional argument for encryption type, can be either pyDes.ECB
#	(Electronic Code Book), pyDes.CBC (Cypher Block Chaining)
#IV   -> Optional Initial Value bytes, must be supplied if using CBC mode.
#	 Must be 8 bytes in length.
#pad  -> Optional argument, set the pad character (PAD_NORMAL) to use
#	 during all encrypt/decrypt operations done with this instance.
#padmode -> Optional argument, set the padding mode (PAD_NORMAL or
#	 PAD_PKCS5) to use during all encrypt/decrypt operations done
#	 with this instance.
data = input("Please enter plain text for encryption:")
keystring=input("Enter 16/24 byte string for key generation:")
k = pyDes.triple_des(keystring, padmode=pyDes.PAD_PKCS5)
e = k.encrypt(data)
print ("cipher text: %r" % e)
#%r to be useful for printing a string of unknown encoding
print ("plain text: %r" % k.decrypt(e))
t1 = time()
for i in range(1000):
     e = k.encrypt(data)
t2 = time()
print("Elapsed time for 1,000 encryptions: {:0.3f}s".format(t2 - t1))


Please enter plain text for encryption:prasann
Enter 16/24 byte string for key generation:1010101010101010
cipher text: b'/\xa4\x86\x8c\x0cg2\xf1'
plain text: b'prasann'
Elapsed time for 1,000 encryptions: 1.622s


In [6]:

from sys import exit
from time import time
 
KeyLength = 10
SubKeyLength = 8
DataLength = 8
FLength = 4
 
# Tables for initial and final permutations (b1, b2, b3, ... b8)
IPtable = (2, 6, 3, 1, 4, 8, 5, 7)
FPtable = (4, 1, 3, 5, 7, 2, 8, 6)
 
# Tables for subkey generation (k1, k2, k3, ... k10)
P10table = (3, 5, 2, 7, 4, 10, 1, 9, 8, 6)
P8table = (6, 3, 7, 4, 8, 5, 10, 9)
 
# Tables for the fk function
EPtable = (4, 1, 2, 3, 2, 3, 4, 1)
S0table = (1, 0, 3, 2, 3, 2, 1, 0, 0, 2, 1, 3, 3, 1, 3, 2)
S1table = (0, 1, 2, 3, 2, 0, 1, 3, 3, 0, 1, 0, 2, 1, 0, 3)
P4table = (2, 4, 3, 1)
 
def perm(inputByte, permTable):
    """Permute input byte according to permutation table"""
    outputByte = 0
    for index, elem in enumerate(permTable):
        if index >= elem:
            outputByte |= (inputByte & (128 >> (elem - 1))) >> (index - (elem - 1))
        else:
            outputByte |= (inputByte & (128 >> (elem - 1))) << ((elem - 1) - index)
    return outputByte
 
def ip(inputByte):
    """Perform the initial permutation on data"""
    return perm(inputByte, IPtable)
 
def fp(inputByte):
    """Perform the final permutation on data"""
    return perm(inputByte, FPtable)
 
def swapNibbles(inputByte):
    """Swap the two nibbles of data"""
    return (inputByte << 4 | inputByte >> 4) & 0xff
 
def keyGen(key):
    """Generate the two required subkeys"""
    def leftShift(keyBitList):
        """Perform a circular left shift on the first and second five bits"""
        shiftedKey = [None] * KeyLength
        shiftedKey[0:9] = keyBitList[1:10]
        shiftedKey[4] = keyBitList[0]
        shiftedKey[9] = keyBitList[5]
        return shiftedKey
 
    # Converts input key (integer) into a list of binary digits
    keyList = [(key & 1 << i) >> i for i in reversed(range(KeyLength))]
    permKeyList = [None] * KeyLength
    for index, elem in enumerate(P10table):
        permKeyList[index] = keyList[elem - 1]
    shiftedOnceKey = leftShift(permKeyList)
    shiftedTwiceKey = leftShift(leftShift(shiftedOnceKey))
    subKey1 = subKey2 = 0
    for index, elem in enumerate(P8table):
        subKey1 += (128 >> index) * shiftedOnceKey[elem - 1]
        subKey2 += (128 >> index) * shiftedTwiceKey[elem - 1]
    return (subKey1, subKey2)
 
def fk(subKey, inputData):
    """Apply Feistel function on data with given subkey"""
    def F(sKey, rightNibble):
        aux = sKey ^ perm(swapNibbles(rightNibble), EPtable)
        index1 = ((aux & 0x80) >> 4) + ((aux & 0x40) >> 5) + \
                 ((aux & 0x20) >> 5) + ((aux & 0x10) >> 2)
        index2 = ((aux & 0x08) >> 0) + ((aux & 0x04) >> 1) + \
                 ((aux & 0x02) >> 1) + ((aux & 0x01) << 2)
        sboxOutputs = swapNibbles((S0table[index1] << 2) + S1table[index2])
        return perm(sboxOutputs, P4table)
 
    leftNibble, rightNibble = inputData & 0xf0, inputData & 0x0f
    return (leftNibble ^ F(subKey, rightNibble)) | rightNibble
 
def encrypt(key, plaintext):
    """Encrypt plaintext with given key"""
    data = fk(keyGen(key)[0], ip(plaintext))
    return fp(fk(keyGen(key)[1], swapNibbles(data)))
 
def decrypt(key, ciphertext):
    """Decrypt ciphertext with given key"""
    data = fk(keyGen(key)[1], ip(ciphertext))
    return fp(fk(keyGen(key)[0], swapNibbles(data)))  
 
if __name__ == '__main__':
    
    try:
        assert encrypt(0b0000000000, 0b10101010) == 0b00010001
    except AssertionError:
        print("Error on encrypt:")
        print("Output: ", encrypt(0b0000000000, 0b10101010), "Expected: ", 0b00010001)
        exit(1)
    try:
        assert encrypt(0b1110001110, 0b10101010) == 0b11001010
    except AssertionError:
        print("Error on encrypt:")
        print("Output: ", encrypt(0b1110001110, 0b10101010), "Expected: ", 0b11001010)
        exit(1)
    try:
        assert encrypt(0b1110001110, 0b01010101) == 0b01110000
    except AssertionError:
        print("Error on encrypt:")
        print("Output: ", encrypt(0b1110001110, 0b01010101), "Expected: ", 0b01110000)
        exit(1)
    try:
        assert encrypt(0b1111111111, 0b10101010) == 0b00000100
    except AssertionError:
        print("Error on encrypt:")
        print("Output: ", encrypt(0b1111111111, 0b10101010), "Expected: ", 0b00000100)
        exit(1)
 
    t1 = time()
    for i in range(1000):
        encrypt(0b1110001110, 0b10101010)
    t2 = time()
    print("Elapsed time for 1,000 encryptions: {:0.3f}s".format(t2 - t1))



Elapsed time for 1,000 encryptions: 0.038s


## AES

In [8]:
!pip install pycrypto

     |████████████████████████████████| 446 kB 6.7 MB/s 
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp37-cp37m-linux_x86_64.whl size=499932 sha256=185e543e82688d2f4d3d8cd5a5cac1da981a8188eb558d45445597a8bb6ba7fc
  Stored in directory: /root/.cache/pip/wheels/cf/85/ba/bbd7c96add459de7598fb424e5ff2309baf2095c844ac0f191
Successfully built pycrypto


In [9]:
# python3 aes.py
from Crypto.Cipher import AES

#key has to be either 16, 24 or 32 bytes
def paddedKey(key):
    while len(key) % 8 !=0:
        key +=' '
    return key
    
#text has to in multiples of 26 bytes
def paddedText(text):
    while len(text) % 16 != 0:
        text += ' '
    return text
        
plain_input = input("Enter the text to be encrypted: ")
plain = paddedText(plain_input)
 
key_input = input("Enter in a key between 16 and 32 characters: ")
key = paddedKey(key_input)
 
if(len(key_input)< 16 & len(key_input)> 32):
 print("key must be between 16 and 32 characters!")
 
cipher = AES.new(key)
ciphertext = cipher.encrypt(plain)
print(ciphertext)
 
cleartext = cipher.decrypt(ciphertext)
print(cleartext)


Enter the text to be encrypted: prasann
Enter in a key between 16 and 32 characters: 1010101010101010
b'[s\x870\xc6\xb0\x84\x13.\xbf+\xb9\xa5\xe2\xe9\xf7'
b'prasann         '


In [12]:
# Description: Simplified AES implementation in Python 3
import sys
 
# S-Box
sBox  = [0x9, 0x4, 0xa, 0xb, 0xd, 0x1, 0x8, 0x5,
         0x6, 0x2, 0x0, 0x3, 0xc, 0xe, 0xf, 0x7]
 
# Inverse S-Box
sBoxI = [0xa, 0x5, 0x9, 0xb, 0x1, 0x7, 0x8, 0xf,
         0x6, 0x0, 0x2, 0x3, 0xc, 0x4, 0xd, 0xe]
 
# Round keys: K0 = w0 + w1; K1 = w2 + w3; K2 = w4 + w5
w = [None] * 6
 
def mult(p1, p2):
    """Multiply two polynomials in GF(2^4)/x^4 + x + 1"""
    p = 0
    while p2:
        if p2 & 0b1:
            p ^= p1
        p1 <<= 1
        if p1 & 0b10000:
            p1 ^= 0b11
        p2 >>= 1
    return p & 0b1111
 
def intToVec(n):
    """Convert a 2-byte integer into a 4-element vector"""
    return [n >> 12, (n >> 4) & 0xf, (n >> 8) & 0xf,  n & 0xf]            
 
def vecToInt(m):
    """Convert a 4-element vector into 2-byte integer"""
    return (m[0] << 12) + (m[2] << 8) + (m[1] << 4) + m[3]
 
def addKey(s1, s2):
    """Add two keys in GF(2^4)"""  
    return [i ^ j for i, j in zip(s1, s2)]
     
def sub4NibList(sbox, s):
    """Nibble substitution function"""
    return [sbox[e] for e in s]
     
def shiftRow(s):
    """ShiftRow function"""
    return [s[0], s[1], s[3], s[2]]
 
def keyExp(key):
    """Generate the three round keys"""
    def sub2Nib(b):
        """Swap each nibble and substitute it using sBox"""
        return sBox[b >> 4] + (sBox[b & 0x0f] << 4)
 
    Rcon1, Rcon2 = 0b10000000, 0b00110000
    w[0] = (key & 0xff00) >> 8
    w[1] = key & 0x00ff
    w[2] = w[0] ^ Rcon1 ^ sub2Nib(w[1])
    w[3] = w[2] ^ w[1]
    w[4] = w[2] ^ Rcon2 ^ sub2Nib(w[3])
    w[5] = w[4] ^ w[3]
 
def encrypt(ptext):
    """Encrypt plaintext block"""
    def mixCol(s):
        return [s[0] ^ mult(4, s[2]), s[1] ^ mult(4, s[3]),
                s[2] ^ mult(4, s[0]), s[3] ^ mult(4, s[1])]    
     
    state = intToVec(((w[0] << 8) + w[1]) ^ ptext)
    state = mixCol(shiftRow(sub4NibList(sBox, state)))
    state = addKey(intToVec((w[2] << 8) + w[3]), state)
    state = shiftRow(sub4NibList(sBox, state))
    return vecToInt(addKey(intToVec((w[4] << 8) + w[5]), state))
     
def decrypt(ctext):
    """Decrypt ciphertext block"""
    def iMixCol(s):
        return [mult(9, s[0]) ^ mult(2, s[2]), mult(9, s[1]) ^ mult(2, s[3]),
                mult(9, s[2]) ^ mult(2, s[0]), mult(9, s[3]) ^ mult(2, s[1])]
     
    state = intToVec(((w[4] << 8) + w[5]) ^ ctext)
    state = sub4NibList(sBoxI, shiftRow(state))
    state = iMixCol(addKey(intToVec((w[2] << 8) + w[3]), state))
    state = sub4NibList(sBoxI, shiftRow(state))
    return vecToInt(addKey(intToVec((w[0] << 8) + w[1]), state))
 
if __name__ == '__main__':
    
     
    plaintext = 0b1101011100101000
    key = 0b0100101011110101
    ciphertext = 0b0010010011101100
    keyExp(key)
    try:
        assert encrypt(plaintext) == ciphertext
    except AssertionError:
        print("Encryption error")
        print(encrypt(plaintext), ciphertext)
        sys.exit(1)
    try:
        assert decrypt(ciphertext) == plaintext
    except AssertionError:
        print("Decryption error")
        print(decrypt(ciphertext), plaintext)
        sys.exit(1)
    print("Test ok!")

Test ok!


## Diffie Hellman

In [13]:
#  7 11 3 6
#Diffie Hellman Key Exchange Alogorithm
# Begin
sharedPrime = int(input("Enter shared Prime(n):"))
sharedBase = int(input("Enter shared Base(g):"))
aliceSecret = int(input("Enter Alice Secret Key(x):"))
bobSecret = int(input("Enter Bob Secret Key(y):"))
print( "Publicly Shared Variables:")
print( "    Publicly Shared Prime: " , sharedPrime )
print( "    Publicly Shared Base:  " , sharedBase )
 
# Alice Sends Bob A = g^x mod n
A = (sharedBase**aliceSecret) % sharedPrime
print( "\n  Alice Sends(A) Over Public Chanel: " , A )
 
# Bob Sends Alice B = g^y mod n
B = (sharedBase ** bobSecret) % sharedPrime
print("Bob Sends(B) Over Public Chanel: ", B )
 
print( "\n------------\n" )
print( "Privately Calculated Shared Secret:" )
# Alice Computes Shared Secret: K1 = B^x mod n
aliceSharedSecret = (B ** aliceSecret) % sharedPrime
print( "    Alice Shared Secret: ", aliceSharedSecret )
 
# Bob Computes Shared Secret: K2 = A^y mod n
bobSharedSecret = (A**bobSecret) % sharedPrime
print( "    Bob Shared Secret: ", bobSharedSecret )


Enter shared Prime(n):7
Enter shared Base(g):11
Enter Alice Secret Key(x):3
Enter Bob Secret Key(y):6
Publicly Shared Variables:
    Publicly Shared Prime:  7
    Publicly Shared Base:   11

  Alice Sends(A) Over Public Chanel:  1
Bob Sends(B) Over Public Chanel:  1

------------

Privately Calculated Shared Secret:
    Alice Shared Secret:  1
    Bob Shared Secret:  1


## RSA

In [14]:
# 17, 13, 5

def gcd(a, b): # calculates GCD of a and d
    while b != 0:
        c = a % b
        a = b
        b = c
    return a


def modinv(a, m): # calculates modulo inverse of a for mod m
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None


def coprimes(a): # calculates all possible co-prime numbers with a
    l = []
    for x in range(2, a):
        if gcd(a, x) == 1 and modinv(x, phi) != None:
            l.append(x)
    for x in l:
        if x == modinv(x, phi):
            l.remove(x)
    return l

def encrypt_block(m): # encrypts a single block
    c = m ** e % n
    return c


def decrypt_block(c): # decrypts a single block
    m = c ** d % n
    return m


def encrypt_string(s): # applies encryption
    return ''.join([chr(encrypt_block(ord(x))) for x in list(s)])


def decrypt_string(s): # applies decryption
    return ''.join([chr(decrypt_block(ord(x))) for x in list(s)])

if __name__ == "__main__":
    p = int(input('Enter prime p: '))
    q = int(input('Enter prime q: '))

    print("Choosen primes:\np=" + str(p) + ", q=" + str(q) + "\n")

    n = p * q
    print("n = p * q = " + str(n) + "\n")

    phi = (p - 1) * (q - 1)
    print("Euler's function (totient) [phi(n)]: " + str(phi) + "\n")

    print("Choose an e from a below coprimes array:\n")
    print(str(coprimes(phi)) + "\n")
    e = int(input())

    d = modinv(e, phi) # calculates the decryption key d

    print("\nYour public key is a pair of numbers (e=" + str(e) + ", n=" + str(n) + ").\n")
    print("Your private key is a pair of numbers (d=" + str(d) + ", n=" + str(n) + ").\n")

    s = input("Enter a message to encrypt: ")
    print("\nPlain message: " + str(s) + "\n")
    enc = encrypt_string(s)
    print("Encrypted message: ", enc, "\n")
    dec = decrypt_string(enc)
    print("Decrypted message: " + dec + "\n")
                        


Enter prime p: 17
Enter prime q: 13
Choosen primes:
p=17, q=13

n = p * q = 221

Euler's function (totient) [phi(n)]: 192

Choose an e from a below coprimes array:

[5, 7, 11, 13, 17, 19, 23, 25, 29, 35, 37, 41, 43, 47, 49, 53, 55, 59, 61, 67, 71, 73, 77, 79, 83, 85, 89, 91, 97, 101, 103, 107, 109, 113, 115, 119, 121, 125, 131, 133, 137, 139, 143, 145, 149, 151, 155, 157, 163, 167, 169, 173, 175, 179, 181, 185, 187]

5

Your public key is a pair of numbers (e=5, n=221).

Your private key is a pair of numbers (d=77, n=221).

Enter a message to encrypt: prasann

Plain message: prasann

Encrypted message:  }­6b6 

Decrypted message: prasann

